In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('matrix_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'matrix_df.csv'

In [ ]:
data = data.drop('data_type', axis=1)

In [ ]:
data.head()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix

# Assuming 'data' is your DataFrame
X = data.drop('suicidal', axis=1)
y = data['suicidal']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing models with a focus on recall
models = {
    'Dummy Classifier': DummyClassifier(strategy='most_frequent'),
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced'),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC(probability=True, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier()
}

# DataFrame to store results
results_df = pd.DataFrame()

# Training and predicting with each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', zero_division=0)
    roc_auc = roc_auc_score(y_test, y_prob)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    tpr = tp / (tp + fn)  # True Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    fpr = fp / (tn + fp)  # False Positive Rate
    fnr = fn / (tp + fn)  # False Negative Rate

    # Creating a temporary DataFrame to hold current results
    temp_df = pd.DataFrame({
        "Model": [name],
        "Accuracy": [acc],
        "Precision": [prec],
        "Recall": [recall],
        "F1-Score": [fscore],
        "ROC AUC": [roc_auc],
        "TPR": [tpr],
        "TNR": [tnr],
        "FPR": [fpr],
        "FNR": [fnr]
    })

    # Concatenating with the main DataFrame
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

In [ ]:
# Display the DataFrame
print(results_df)

                 Model  Accuracy  Precision    Recall  F1-Score   ROC AUC  \
0     Dummy Classifier  0.664289   0.000000  0.000000  0.000000  0.500000   
1  Logistic Regression  0.977263   0.968000  0.964143  0.966068  0.987943   
2        Decision Tree  0.968346   0.958333  0.946879  0.952572  0.972871   
3                  SVM  0.971021   0.964865  0.948207  0.956463  0.983592   
4        Random Forest  0.973696   0.961436  0.960159  0.960797  0.986116   
5    Gradient Boosting  0.977708   0.976934  0.956175  0.966443  0.987493   

        TPR       TNR       FPR       FNR  
0  0.000000  1.000000  0.000000  1.000000  
1  0.964143  0.983893  0.016107  0.035857  
2  0.946879  0.979195  0.020805  0.053121  
3  0.948207  0.982550  0.017450  0.051793  
4  0.960159  0.980537  0.019463  0.039841  
5  0.956175  0.988591  0.011409  0.043825  


In [ ]:
results_df.to_csv("simple_model_performance.csv", index=False)